In [ ]:
import requests
from bs4 import BeautifulSoup

In [63]:
class BillFinder:
    
    def __init__(self):
        
        self.source_urls_energy = {'Canada': 'https://en.wikipedia.org/wiki/List_of_Canadian_electric_utilities',
                                   'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_electric_companies',
                                   'UK': 'https://www.utilitysavingexpert.com/energy/suppliers/'}
        
        self.source_urls_water = {'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_water_companies',
                                  'UK': 'https://en.wikipedia.org/wiki/United_Kingdom_water_companies'}
    
    def get_energy_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls_energy[country]).text)
        
        if country == 'Canada':
            for t in soup.find_all('table', role="presentation"):
                for _ in t.find_all('li'):
                    for a in _.find_all('a'):
                        company_set.add(a.text)
        elif country == 'USA':
            for t in soup.find('span', id="List_of_US_electric_companies_by_state"):
                for e in t.parent.next_elements:
                    if e.name == 'table':
                        break
                    if e.name == 'ul':
                        for _ in e.find_all('li'):
                            if company_name := _.text.strip():
                                company_set.add(company_name)
        elif country == 'UK':
            for header_text in ['List of UK energy suppliers', 'List of business energy suppliers']:
                p = soup.find('h2', string=header_text).parent
                for _ in p.find_all('div', class_="table-responsive"):
                    for row in _.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[1].text)
                            
        print(f'found {len(company_set):,} companies')
                        
        return company_set
    
    def get_water_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls_water[country]).text)
        
        if country == 'USA':
            for t in soup.find_all('h2'):
                if t.find_all('a', href=True):
                    if list_of_company_names := t.find_next_sibling('ul'):
                        for name_line in list_of_company_names.find_all('li'):
                            if company_name := name_line.text.strip():
                                company_set.add(company_name)
        elif country == 'UK':
            for s in soup.find('span', id='Water_and_sewerage').parent.next_siblings:
                if s.name == 'table':
                    for row in s.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[0].text.strip()) 
                    break
            for s in soup.find('span', id='Water_only').parent.next_siblings:
                if s.name == 'table':
                    for row in s.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[0].text.strip()) 
                    break
                    
            for id_ in ['Scotland', 'Northern_Ireland', 'Crown_dependencies']:
                for s in soup.find('span', id=id_).parent.next_siblings:
                    if s.name == 'ul':
                        for item_ in s.find_all('li'):
                            company_set.add(item_.text)
                        break
                            
        print(f'found {len(company_set):,} companies')
                        
        return company_set

In [64]:
bf = BillFinder()

In [65]:
bf.get_water_companies(country='UK')

found 30 companies


{'Affinity Water',
 'Albion Water',
 'Anglian Water',
 'Bournemouth Water',
 'Bristol Water',
 'Cambridge Water Company',
 'Cholderton and District Water Company',
 'Dŵr Cymru Welsh Water',
 'Essex and Suffolk Water',
 'Everflow[2]',
 'Guernsey Water (government)',
 'Hafren Dyfrdwy',
 'Hartlepool Water',
 'Isle of Man Water Authority (government)',
 'Jersey Water (private)',
 'Northern Ireland Water (government)',
 'Northumbrian Water',
 'Portsmouth Water',
 'Scottish Water (government)',
 'Severn Trent Water',
 'South East Water',
 'South Staffordshire Water',
 'South West Water',
 'Southern Water',
 'Sutton and East Surrey Water',
 'Thames Water',
 'United Utilities',
 'Wessex Water',
 'Yorkshire Water',
 'Youlgrave Waterworks'}